In [2]:
import os
import chemprop
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.decomposition import PCA
import scipy 
import glob


In [5]:
regression_csvs = [ '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/bbb_training.csv', '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/log_s_training.csv']

In [6]:
classification_csvs = ['/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/cyp2d6_inhibitor_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/cyp3a4_substrate_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/pgp_inhibitor_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/cyp3a4_inhibitor_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/oct2_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/cyp2c19_inhibitor_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/caco_2_classification_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/cyp1a2_inhibitor_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/pgp_substrate_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/cyp2c9_inhibitor_training.csv',
 '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/cyp2d6_substrate_training.csv']


### Regression

In [7]:

# training_csv = '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/log_s_training.csv'
# test_csv = '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/log_s_test.csv'
# training_csvs =  glob.glob('/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/*training.csv')
for training_csv in regression_csvs:
    save_dir = os.path.join('/home/ymyung/projects/deeppk/3_Results/pkCSM_vs_interpretable_ADMET','MPNN/',os.path.basename(training_csv).split('.csv')[0])
    test_csv = training_csv.split('_training.csv')[0]+'_test.csv'
    arguments = [
    '--data_path', training_csv,
    '--target_columns','label',
    '--dataset_type', 'regression',
    '--save_dir', save_dir,
    '--epochs', '100',
    '--split_type','cv',
    '--num_folds','3',
    '--save_smiles_splits',
    '--quiet',
    '--show_individual_scores',
    '--metric', 'rmse',
    '--extra_metrics', 'r2', 'mse','mae',
    '--save_preds',
    # '--ensemble_size', '5',
    '--gpu',0,
    '--features_generator','rdkit_2d_normalized', # additional
    '--no_features_scaling' # additional    
    ]

    args = chemprop.args.TrainArgs().parse_args(arguments)
    mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

    arguments = [
    '--test_path', test_csv,
    '--preds_path', '{}/test_preds_cla.csv'.format(save_dir),
    '--checkpoint_dir', save_dir,
    '--features_generator','rdkit_2d_normalized', # additional
    '--no_features_scaling' # additional
    ]

    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)

    # df = pd.read_csv('{}/fold_0/test_full.csv'.format(save_dir))
    # df['preds'] = [x[0] for x in preds]
 

13976it [00:00, 394463.16it/s]
  3%|▎         | 407/13976 [00:15<08:11, 27.59it/s]ERROR:root:Could not compute PEOE_VSA12 for molecule
Traceback (most recent call last):
  File "/home/ymyung/miniconda3/envs/chemprop/lib/python3.10/site-packages/numpy/core/fromnumeric.py", line 40, in _wrapit
    wrap = obj.__array_wrap__
AttributeError: 'float' object has no attribute '__array_wrap__'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ymyung/miniconda3/envs/chemprop/lib/python3.10/site-packages/descriptastorus/descriptors/rdNormalizedDescriptors.py", line 62, in applyNormalizedFunc
    return cdfs[name](rdDescriptors.applyFunc(name,m))
  File "/home/ymyung/miniconda3/envs/chemprop/lib/python3.10/site-packages/descriptastorus/descriptors/rdNormalizedDescriptors.py", line 49, in cdf
    v = dist.cdf(np.clip(v, minV, maxV), loc=loc, scale=scale, *arg)
  File "<__array_function__ internals>", line 180, in clip
  File "/hom

### Classification

In [2]:

# training_csv = '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/caco_2_classification_training.csv'
# test_csv = '/home/ymyung/projects/deeppk/1_dataset/pk_data/processed_data/pkcsm_vs_interpretable_admet/caco_2_classification_test.csv'
for training_csv in classification_csvs:
    save_dir = os.path.join('/home/ymyung/projects/deeppk/3_Results/pkCSM_vs_interpretable_ADMET','MPNN/',os.path.basename(training_csv).split('.csv')[0])
    test_csv = training_csv.split('_training.csv')[0]+'_test.csv'

    arguments = [
    '--data_path', training_csv,
    '--target_columns','label',
    '--dataset_type', 'classification',
    '--save_dir', save_dir,
    '--epochs', '100',
    '--split_type','scaffold_balanced',
    # '--num_folds','3',
    '--save_smiles_splits',
    '--quiet',
    '--show_individual_scores',
    '--metric','mcc',
    '--extra_metrics', 'f1','auc','accuracy',
    '--save_preds',
    '--separate_test_path',test_csv,
    # '--ensemble_size', '5',
    '--gpu',0,
    '--features_generator','rdkit_2d_normalized', # additional
    '--no_features_scaling' # additional    
    ]

    args = chemprop.args.TrainArgs().parse_args(arguments)
    mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

    arguments = [
    '--test_path', test_csv,
    '--preds_path', '{}/test_preds_cla.csv'.format(save_dir),
    '--checkpoint_dir', save_dir,
    '--features_generator','rdkit_2d_normalized', # additional
    '--no_features_scaling' # additional
    ]

    args = chemprop.args.PredictArgs().parse_args(arguments)
    preds = chemprop.train.make_predictions(args=args)

    # df = pd.read_csv('{}/fold_0/test_full.csv'.format(save_dir))
    # df['preds'] = [x[0] for x in preds]
 

650it [00:00, 330180.16it/s]
100%|██████████| 650/650 [00:00<00:00, 128356.76it/s]
Fold 0
1411it [00:00, 327785.26it/s]
100%|██████████| 650/650 [00:00<00:00, 5133.99it/s]
650it [00:00, 390195.73it/s]
100%|██████████| 100/100 [00:43<00:00,  2.32it/s]
Model 0 best validation mcc = 0.568823 on epoch 83
Model 0 test mcc = 0.160738                    
Model 0 test label mcc = 0.160738
Model 0 test f1 = 0.445352
Model 0 test label f1 = 0.445352
Model 0 test auc = 0.627835
Model 0 test label auc = 0.627835
Model 0 test accuracy = 0.615167
Model 0 test label accuracy = 0.615167
Ensemble test mcc = 0.160738
Ensemble test label mcc = 0.160738
Ensemble test f1 = 0.445352
Ensemble test label f1 = 0.445352
Ensemble test auc = 0.627835
Ensemble test label auc = 0.627835
Ensemble test accuracy = 0.615167
Ensemble test label accuracy = 0.615167
1-fold cross validation
	Seed 0 ==> test mcc = 0.160738
		Seed 0 ==> test label mcc = 0.160738
	Seed 0 ==> test f1 = 0.445352
		Seed 0 ==> test label f1 = 0.4

Loading training args
Setting molecule featurization parameters to default.
Loading data


1411it [00:00, 422605.18it/s]
100%|██████████| 1411/1411 [01:00<00:00, 23.15it/s]


Validating SMILES
Test size = 1,411


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda


100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


Saving predictions to /home/ymyung/projects/deeppk/3_Results/pkCSM_vs_interpretable_ADMET/MPNN/caco_2_classification_training/test_preds_cla.csv
Elapsed time = 0:01:02
